# Objetivo:

- Carregar os dados da camada Silver (Empresas e Sócios), com o Polars (execução mais rápida do que o Pandas);
- Aplicar as regras de negócio nas tabelas (Quantidade de sócios, Sócios Estrangeiros, doc_alvo);
- Salva o resultado na Camada Gold.

### Import das Bibliotecas

In [ ]:
import polars as pl
import os

### Definição dos Diretórios

In [ ]:
DATA_DIR = '../data'

SILVER_DIR = os.path.join(DATA_DIR, 'silver')
GOLD_DIR = os.path.join(DATA_DIR, 'gold')

# Garante que os diretórios existam
os.makedirs(SILVER_DIR, exist_ok=True)
os.makedirs(GOLD_DIR, exist_ok=True)

### Definição dos nomes dos Arquivos na camada Silver Dinâmico

In [ ]:
EMPRESA_FILE_SILVER = None
SOCIO_FILE_SILVER = None

# Lista todos os arquivos no diretório silver

try:
    silver_files = os.listdir(SILVER_DIR)

    # Procura pelos arquivos de Empresa e Socio com base no prefixo
    for filename in silver_files:
        if filename.startswith('empresas'):
            EMPRESA_FILE_SILVER = os.path.join(SILVER_DIR, filename)
        elif filename.startswith('socios'):
            SOCIO_FILE_SILVER = os.path.join(SILVER_DIR, filename)

    # Verifica se os arquivos foram encontrados
    if EMPRESA_FILE_SILVER is None:
        raise FileNotFoundError(f"Nenhum arquivo começando com 'empresas' encontrado em {SILVER_DIR}. Verifique o nome do arquivo.")
    if SOCIO_FILE_SILVER is None:
        raise FileNotFoundError(f"Nenhum arquivo começando com 'socios' encontrado em {SILVER_DIR}. Verifique o nome do arquivo.")

except FileNotFoundError as e:
    print(f"ERRO: {e}")
    exit()
except Exception as e:
    print(f"Ocorreu um erro ao listar arquivos no diretório silver: {e}")
    exit()

#### Nome do Arquivo na Camada Gold

In [ ]:
FILE_GOLD = os.path.join(GOLD_DIR, 'gold.parquet')

### Carregar dados da Camada Silver

#### Leitura Tabela Empresas

In [ ]:
# Ler tabela Empresas

df_empresas = pl.read_parquet(EMPRESA_FILE_SILVER)
df_empresas.schema

#### Leitura Tabela Sócios

In [ ]:
# Ler tabela Sócios

df_socios = pl.read_parquet(SOCIO_FILE_SILVER)
df_socios.schema

### Tratamentos

#### Aplicar regra de negócio nos Sócios

In [ ]:
socios_aggs = (
    df_socios
    .group_by("cnpj")
    .agg([
        pl.count("nome_socio").alias("qtde_socios"),
        pl.col("documento_socio")
          .cast(pl.Utf8)
          .str.starts_with("999")
          .max()
          .alias("flag_socio_estrangeiro")
    ])
)

#### Juntar Empresas com Sócios

In [ ]:
df_gold = (
    df_empresas
    .join(socios_aggs, on="cnpj", how="left")
    .with_columns([
        ((pl.col("cod_porte") == "03") & (pl.col("qtde_socios") > 1)).alias("doc_alvo")
    ])
)

### Salvar na Camada Gold

In [ ]:
df_gold.write_parquet(FILE_GOLD)

print("Camada Gold gerada com sucesso:", df_gold.shape)
df_gold.head(50)